In [1]:
#Q1 --Model Building  import libraries


# Installation for GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.28 --force-reinstall --upgrade --no-cache-dir --verbose

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 30.1 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.1/152.1 kB 4.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Running command Getting requirements to build wheel
  Could not determine CMake version via --version, got '' 'Traceback (most recent call last):\n  File "/usr/local/bin/cmake", line 5, in <module>\n    from cmake import cmake\nModuleNotFoundError: No module named \'cmake\'\n'
  Getting requirements to build wheel ... done
  Running command pip subprocess to install backend dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━

**Q1: Model Building**

Write Python code that imports the 'hf_hub_download' function from the 'huggingface_hub' library and also imports the 'Llama' class from the 'llama_cpp' library. Ensure that the code is correctly formatted and ready for execution. Details : - Import the 'hf_hub_download' function from the 'huggingface_hub' library - Import the 'Llama' class from the 'llama_cpp' library - Define the model name or path as a string (You can find this info from hugging face website) - Define the model basename as a string, indicating it's in the gguf format - # Download the model from the Hugging Face Hub using the 'hf_hub_download' function by specifying the 'repo_id' and 'filename' - Create an instance of the 'Llama' class with specified parameters - remove the blank spaces and complete the code

In [2]:
# used to supress display of warnings
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

# display all dataframe columns
pd.options.display.max_columns = None

# display all dataframe rows
pd.options.display.max_rows = None

pd.options.display.max_colwidth=1500
pd.options.display.max_info_rows = 100

In [3]:
pip install huggingface_hub

In [4]:
from huggingface_hub import hf_hub_download

In [2]:
#!pip install nvidia-cuda-toolkit

ERROR: Could not find a version that satisfies the requirement nvidia-cuda-toolkit (from versions: none)
ERROR: No matching distribution found for nvidia-cuda-toolkit


In [3]:
#!ldconfig -p | grep libcuda.so.1

In [5]:
from llama_cpp import Llama

In [6]:
 # Model configuration
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_M.gguf"
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
    )

llama-2-13b-chat.Q5_K_M.gguf:   0%|          | 0.00/9.23G [00:00<?, ?B/s]

In [7]:
lcpp_llm = Llama(
        model_path=model_path,
        n_threads=2,  # CPU cores
        n_batch=512,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
        n_gpu_layers=43,  # Change this value based on your model and your GPU VRAM pool.
        n_ctx=4096,  # Context window
    )

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


**Q2 - Create System Message**

Define the System Message - The system message must start with a line defining what the bot's identity is. - The next line in the system prompt must have a clear description of what the bot is expected to do according to the business objective - There are 5 structured fields expected in the output - 1) Categories - (Technical issue, Hardware issue, Data Recovery)  2) Tags  3) Priority  4) Suggest ETA  5) Generated 1st Reply (Sentiment-Based)

**Zero Shot prompting**

In [12]:
prompt_template = """<s>[INST]\n <<SYS>> \n {system_message} \n <</SYS>>```{user_message}``` /n [/INST] """

In [13]:
system_message = """You are Alex a technical assistant specializing in customer input analysis.
your goal is to extract specific information from customer support tickets.
You will be provided with user complaints.
Your task is do the below activities step by step,

strictly provide all the output in the form of a JSON object containing the following information from customer support tickets:

1. Ticket Category: Identify whether the support ticket falls under 'Technical issues',
   'Hardware issues', or 'Data recovery' category based on the provided Support Ticket Text. (as a string)
2. Tags: Generate tags to assist in classifying the ticket. (as an  string)
3. Priority: Determine priority as 'High' or 'Normal' based on the content of the text."(as an  string)
4. ETA : Propose an estimated time of arrival for resolving the issue mentioned in the ticket text.(as an integer)
5. response : Craft a polite response based on the sentiment expressed in the ticket. (as a string)

"""

In [14]:
def generate_prompt(system_message,user_input):
    prompt=prompt_template.format(system_message=system_message,user_message=user_input)
    return prompt

In [15]:
def generate_llama_response(user_message):
    #create prompt
    prompt=generate_prompt(system_message,user_message)
    # Generate a response from the LLaMA model
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=1200,
        temperature=0,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=['/s'],
        echo=False
    )

    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    return response_text

# Sample Response:

In [12]:
user_complaint='''I accidentally formatted my USB drive with crucial files. Help needed for data recovery.'''

In [13]:
response = generate_llama_response(user_complaint)
print(response)

 Sure, I'd be happy to help you extract specific information from customer support tickets! Based on the user complaint you provided, here is the output in JSON object format:

{
"Ticket Category": "Data Recovery",
"Tags": "USB drive, data recovery, files",
"Priority": "High",
"ETA": 3,
"Response": "Hi there! Sorry to hear that you accidentally formatted your USB drive with crucial files. Our team will work on recovering your data as soon as possible. Please provide us with the details of the situation and we'll keep you updated on our progress."
}

Here's how I arrived at each field in the JSON object:

1. Ticket Category: Based on the user complaint, this ticket falls under the "Data Recovery" category.
2. Tags: The tags generated for this ticket are "USB drive", "data recovery", and "files". These tags will help classify the ticket and make it easier to search for similar tickets in the future.
3. Priority: Based on the urgency of the issue (the user needs their crucial files recove

**Q3 - Loading the dataset**

In [162]:
import json
##COlab Drive mount
from google.colab import drive
import os
import pandas as pd

In [163]:
drive.mount('/content/drive')

Mounted at /content/drive


In [164]:
os.chdir('/content/drive/MyDrive/LLM/Week-6-Project/')

In [44]:
user_complaints_df = pd.read_csv('Support_ticket_text_data_mid_term.csv')

In [45]:
user_complaints_df.head(5)

,support_tick_id,support_ticket_text
0,ST2023-006,"My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly."
1,ST2023-007,"Urgent help required! My laptop refuses to start, and I have a crucial presentation scheduled for tomorrow. I've attempted a restart, but it hasn't worked. Please provide immediate assistance to resolve this hardware issue"
2,ST2023-008,"I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?"
3,ST2023-009,"Despite being in close proximity to my Wi-Fi router, the signal remains persistently weak in my home. This issue has been ongoing, and I need assistance troubleshooting it. Please help me resolve the weak Wi-Fi signal problem."
4,ST2023-010,"My smartphone battery is draining rapidly, even with minimal use. Can you help me identify and rectify this battery issue?"


In [46]:
user_complaints_df.shape

(21, 2)

**Q4 - Apply the model to the dataset**

Create a new column in the DataFrame called 'llama_response' and populate it with responses generated by applying the 'generate_llama_response' function to each 'support_ticket_text' in the DataFrame Details - Apply a function to each element in the 'support_ticket_text' column of the DataFrame 'data' - The applied function, in this case, is a lambda function. - The lambda function takes a single argument 'x', representing each individual 'support_ticket_text' in the column. Inside the lambda function: - 'generate_llama_response(x)' is called to generate a response based on the 'support_ticket_text'. - The result of 'generate_llama_response(x)' is assigned to a new column called 'llama_response' in the DataFrame 'data'. Example - data['llama_response'] = data['support_ticket_text'].apply(lambda x: generate_llama_response(x))

In [21]:
# Constructing the response for all the records/complaints in the input dataset.

user_complaints_df['llama_response'] = user_complaints_df['support_ticket_text'].apply(lambda x: generate_llama_response(x))

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


# Removing the JSON message response text

In [24]:
def parse_json_garbage(s):
    print(s)
    s = s[next(idx for idx, c in enumerate(s) if c in "{["):]
    try:
        return json.loads(s)
    except json.JSONDecodeError as e:
        return json.loads(s[:e.pos])

In [28]:
user_complaints_df['purified_llama_response'] = user_complaints_df['llama_response'].apply(lambda x: x.replace("\n",''))

# There are few responses not been in JSON format, Doing some cleaning activity to convert as JSON object.

In [50]:
user_complaints_df['purified_llama_response'][8]

' Sure, I can help you with that! Based on the support ticket text you provided, here is the information I was able to extract:{"Ticket Category":" Data recovery","Tags":" External hard drive, not recognized, vital data, recovery","Priority":" High","ETA":" 2-3 business days (assuming no further complications)","Response":" Dear [Customer], thank you for reaching out to us regarding the issue with your external hard drive. We understand the importance of recovering your vital data and will do our best to assist you. Our team will work on resolving this matter as soon as possible, and we estimate that it may take 2-3 business days to complete. Please let us know if there is anything else we can do to help in the meantime.Please note that these are just estimates based on the information provided, and actual resolution times may vary depending on the complexity of the issue and other factors."}'

In [33]:
user_complaints_df['purified_llama_response'][8] = user_complaints_df['purified_llama_response'][8].replace("**Ticket Category","{\"Ticket Category\"")

In [35]:
user_complaints_df['purified_llama_response'][8] = user_complaints_df['purified_llama_response'][8].replace("Tags",",\"Tags\"")

In [37]:
user_complaints_df['purified_llama_response'][8] = user_complaints_df['purified_llama_response'][8].replace("Priority",",\"Priority\"")

In [38]:
user_complaints_df['purified_llama_response'][8] = user_complaints_df['purified_llama_response'][8].replace("ETA",",\"ETA\"")

In [39]:
user_complaints_df['purified_llama_response'][8] = user_complaints_df['purified_llama_response'][8].replace("Response",",\"Response\"")

In [41]:
user_complaints_df['purified_llama_response'][8] = user_complaints_df['purified_llama_response'][8].replace("**","\"")

In [69]:
user_complaints_df['purified_llama_response'][8] = user_complaints_df['purified_llama_response'][8].replace("factors.","factors.\"}")

In [86]:
user_complaints_df['purified_llama_response'][16] = user_complaints_df['purified_llama_response'][16].replace("\n",'')
user_complaints_df['purified_llama_response'][16] = user_complaints_df['purified_llama_response'][16].replace("hours",'')

In [87]:
user_complaints_df['llama_response_parsed'] = user_complaints_df['purified_llama_response'].apply(lambda x: parse_json_garbage(x))


 Sure! Here's the output you requested based on the provided support ticket text:{"Ticket Category": "Technical issues","Tags": "internet connection, slow internet, disconnections","Priority": "High","ETA": 24,"Response": "Dear [Customer], sorry to hear that you're experiencing connectivity issues. Our team is here to help! We understand the inconvenience this has caused and will work promptly to resolve the issue. Can you please provide us with more details about your internet service provider and any error messages you may have received? We'll get back to you as soon as possible with a solution."}
 Sure, I can assist you with that! Based on the support ticket text you provided, here is the information extracted in JSON format:{"Ticket Category": "Hardware Issues","Tags": "Laptop startup issues, urgent assistance","Priority": "High","ETA": 24,"Response": "Dear [Customer], Sorry to hear that your laptop is not starting up. We understand the urgency of the situation and will provide imm

# Below are the Purified Llama response and Parsed Llama response:

In [88]:
user_complaints_df.head(22)

,support_tick_id,support_ticket_text,llama_response,purified_llama_response,llama_response_parsed
0,ST2023-006,"My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly.","Sure! Here's the output you requested based on the provided support ticket text:\n\n{\n""Ticket Category"": ""Technical issues"",\n""Tags"": ""internet connection, slow internet, disconnections"",\n""Priority"": ""High"",\n""ETA"": 24,\n""Response"": ""Dear [Customer], sorry to hear that you're experiencing connectivity issues. Our team is here to help! We understand the inconvenience this has caused and will work promptly to resolve the issue. Can you please provide us with more details about your internet...","Sure! Here's the output you requested based on the provided support ticket text:{""Ticket Category"": ""Technical issues"",""Tags"": ""internet connection, slow internet, disconnections"",""Priority"": ""High"",""ETA"": 24,""Response"": ""Dear [Customer], sorry to hear that you're experiencing connectivity issues. Our team is here to help! We understand the inconvenience this has caused and will work promptly to resolve the issue. Can you please provide us with more details about your internet service provi...","{'Ticket Category': 'Technical issues', 'Tags': 'internet connection, slow internet, disconnections', 'Priority': 'High', 'ETA': 24, 'Response': 'Dear [Customer], sorry to hear that you're experiencing connectivity issues. Our team is here to help! We understand the inconvenience this has caused and will work promptly to resolve the issue. Can you please provide us with more details about your internet service provider and any error messages you may have received? We'll get back to you as so..."
1,ST2023-007,"Urgent help required! My laptop refuses to start, and I have a crucial presentation scheduled for tomorrow. I've attempted a restart, but it hasn't worked. Please provide immediate assistance to resolve this hardware issue","Sure, I can assist you with that! Based on the support ticket text you provided, here is the information extracted in JSON format:\n\n{\n""Ticket Category"": ""Hardware Issues"",\n""Tags"": ""Laptop startup issues, urgent assistance"",\n""Priority"": ""High"",\n""ETA"": 24,\n""Response"": ""Dear [Customer], Sorry to hear that your laptop is not starting up. We understand the urgency of the situation and will provide immediate assistance. Our technical team will work on resolving the hardware issue as soon a...","Sure, I can assist you with that! Based on the support ticket text you provided, here is the information extracted in JSON format:{""Ticket Category"": ""Hardware Issues"",""Tags"": ""Laptop startup issues, urgent assistance"",""Priority"": ""High"",""ETA"": 24,""Response"": ""Dear [Customer], Sorry to hear that your laptop is not starting up. We understand the urgency of the situation and will provide immediate assistance. Our technical team will work on resolving the hardware issue as soon as possible. Pl...","{'Ticket Category': 'Hardware Issues', 'Tags': 'Laptop startup issues, urgent assistance', 'Priority': 'High', 'ETA': 24, 'Response': 'Dear [Customer], Sorry to hear that your laptop is not starting up. We understand the urgency of the situation and will provide immediate assistance. Our technical team will work on resolving the hardware issue as soon as possible. Please expect a resolution within the next 24 hours.'}"
2,ST2023-008,"I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?","Sure! Here is the output in JSON format based on the provided support ticket text:\n\n{\n""Ticket Category"": ""Data Recovery"",\n""Tags"": ""data loss, document recove

**Q5 - Dataset Preparation**

Prepare the dataset in the desired format - Concatenate the original DataFrame 'data' with a new DataFrame created by normalizing JSON data. - The 'data['llama_response_parsed']' column is assumed to contain JSON data that needs to be flattened and normalized. - 'pd.json_normalize' is a pandas function used to normalize semi-structured JSON data into a flat DataFrame. In this case, it's applied to the 'llama_response_parsed' column, which presumably contains JSON data. - The result of 'pd.json_normalize' is a DataFrame where the JSON data is flattened and each element becomes a separate column. - The 'axis=1' parameter specifies that the concatenation should be done horizontally, i.e., the new columns from normalization will be added as new columns in the 'data' DataFrame. - After this operation, the 'data' DataFrame will contain the original columns along with the additional columns generated by normalizing the JSON data from the 'llama_response_parsed - example - data = pd.concat([data, pd.json_normalize(data['llama_response_parsed'])], axis=1) Drop specific columns which are not needed from the DataFrame 'data' Keep the columns that are mentioned in the sample output

In [94]:
final_customer_df = pd.concat([user_complaints_df[["support_tick_id","support_ticket_text"]], pd.json_normalize(user_complaints_df['llama_response_parsed'])], axis=1)

**Below is the extracted details such as Ticket Category, Tags, Priority,ETA in hours and Response from the given Support_Ticket_text.**

In [95]:
final_customer_df

,support_tick_id,support_ticket_text,Ticket Category,Tags,Priority,ETA,Response
0,ST2023-006,"My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly.",Technical issues,"internet connection, slow internet, disconnections",High,24,"Dear [Customer], sorry to hear that you're experiencing connectivity issues. Our team is here to help! We understand the inconvenience this has caused and will work promptly to resolve the issue. Can you please provide us with more details about your internet service provider and any error messages you may have received? We'll get back to you as soon as possible with a solution."
1,ST2023-007,"Urgent help required! My laptop refuses to start, and I have a crucial presentation scheduled for tomorrow. I've attempted a restart, but it hasn't worked. Please provide immediate assistance to resolve this hardware issue",Hardware Issues,"Laptop startup issues, urgent assistance",High,24,"Dear [Customer], Sorry to hear that your laptop is not starting up. We understand the urgency of the situation and will provide immediate assistance. Our technical team will work on resolving the hardware issue as soon as possible. Please expect a resolution within the next 24 hours."
2,ST2023-008,"I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?",Data Recovery,"data loss, document recovery",High,24,Thank you for reaching out to us. We understand the importance of recovering your essential work documents. Our team will prioritize the data recovery process and guide you through it as soon as possible. Please avoid any further actions on your device to prevent any additional damage. We'll keep you updated on the progress.
3,ST2023-009,"Despite being in close proximity to my Wi-Fi router, the signal remains persistently weak in my home. This issue has been ongoing, and I need assistance troubleshooting it. Please help me resolve the weak Wi-Fi signal problem.",Technical issues,"Wi-Fi, connectivity, signal strength",High,2,Thank you for reaching out to us regarding the persistent weak Wi-Fi signal issue in your home. We understand how frustrating this can be and would be happy to help troubleshoot the problem with you. Please provide us with some more information such as your router's model number and any other relevant details so we can assist you further.
4,ST2023-010,"My smartphone battery is draining rapidly, even with minimal use. Can you help me identify and rectify this battery issue?",Hardware Issues,"battery drain, smartphone",High,2,"Thank you for reaching out to us! We're here to help. Based on your description, it sounds like you might be experiencing a hardware issue with your smartphone's battery draining rapidly. To assist you further, could you please provide the following information: - The type of smartphone you are using - The operating system version you have installed - Any recent software updates or changes made to the device? Once we receive this information, our technical team will be happy to help diagnose..."
5,ST2023-011,I'm locked out of my online banking account and need immediate assistance in resetting my password for an urgent transaction.,Technical issues,"password reset, online banking, urgent transaction",High,30,"Dear [Customer], we apologize for the inconvenience you're facing. Our team is working on resolving your issue as soon as possible. Please provide us with the necessary information to reset your password and regain access to your online banking account."
6,ST2023-012,"My computer's performance is sluggish, severely impacting my work. I need help optimizing it to regain productivity.",Technical issues,"performance optimization, slu

# Few Shot Prompting:
  Using Few Shot promt technique We are achiving the same as above.

In [628]:
system_message = """ You are a Customer Support Assistant for ABCDE tech.co.in.
You will be provided with user complaints. User complaints will be delimited by triple backticks, that is, ```.
Your task is to understand the support ticket text completely and generate a response to all the user complaints. Please process the user's complaint,
identify its sentiment, and respond appropriately, always maintaining a polite and helpful tone. If possible, provide relevant information or
troubleshooting steps.

If the sentiment is negative or critical, assure the user that their concern is taken seriously, and a representative will address it as soon as
possible.
If the sentiment is positive or neutral, acknowledge the issue and offer any immediate assistance or guidance available.

output should be in the form of a JSON object for all the given user complaints. Go through the user's feedback step by step, and generate a structured output for further analysis by the customer support assistant
in the below format

{
 "Ticket Category": <Identify whether the support ticket falls under 'Technical issues', 'Hardware issues', or 'Data recovery' category based on the provided Support Ticket Text. (as a string)>,
  "Tags": <Generate tags to assist in classifying the ticket. (as an  string)>,
  "Priority": <Determine priority as 'High' or 'Normal' based on the content of the text.(as a  string)>
  "eta" : <Propose an estimated time of arrival for resolving the issue mentioned in the ticket text.(as a string)>
  "response" : <Craft a polite response based on the sentiment expressed in the ticket. (as a string)>
}

"""

In [629]:
llama_examples_template = """<s>[INST] ```{user_message} ``` [/INST]{assistant_message}</s>"""

In [630]:
first_turn_template = """<s>[INST]{system_message} ``` {user_message} ``` [/INST]{assistant_message}</s> """

In [631]:
llama_prediction_template = """<s>[INST]```{user_message}```[/INST]"""

In [632]:
#creating few shot template prompt
def create_few_shot_prompt(system_message):

    few_shot_prompt = ''
    user_input_1 = """I am facing a serious internet issues. It's been 24 hours and no one has attended my case, all my office work is getting affected highly.Please resolve as soon as possible ."""
    assistant_output_1 ="""{"category": "Technical issues",
                           "tags": ["Internet connection", "Slow internet", "Disconnections"],
                            "priority": "High",
                             "eta": "Within 2 hours",
                            "response": "Thank you for reaching out to us regarding your internet connectivity issues.
                             We apologize for any inconvenience caused by the slow speeds and frequent disconnections.
                             Our team is working diligently to address the matter as soon as possible"}"""

    few_shot_prompt += first_turn_template.format(
                system_message=system_message,
                user_message=user_input_1,
                assistant_message=assistant_output_1)

    user_input_2 =""" my laptop screen got damaged, i am seeingonly the blue screen nothing is coming on my screen."""
    assistant_output_2 = """{"category": "Hardware issues",
                           "tags": ["Laptop issue", "physical damage", "Hardware issues"],
                            "priority": "High",
                             "eta": "Within 7 hours",
                            "response": "Hi there! Sorry to hear that your laptop screen is physically damaged and you are seeing only the blue screen.
                                         Our laptop experts will work on this issue as soon as possible.
                                         Please provide the damaged laptop to our support team for further assistance."} """

    few_shot_prompt += llama_examples_template.format(
                user_message=user_input_2,
                assistant_message=assistant_output_2
            )

    ###Customer complaints with long text
    user_input_3 =""" I hope this message finds you well. I have a serious issue with my laptop, This issue is occurring when i am continuously using my laptop.
                      Large amount of heat is getting discharged and it looks like my laptop fan is not working properly. For example, in recent times whenever i am working on some huge
                      software applications, laptop is getting restarted automatically, which is causing to redo the work. Additionally, i am seeing the bluetooth port is not working properly.
                      I can't able to connect my bluetooth headset to my laptop.These issues are causing significant disruptions to my daily tasks and workflow.While I don't have
                      specific instructions on how to resolve this complex problem,
                      I suspect there may be an underlying issue with the system or software compatibility. Also i am ready to surrender my laptop for complete diagnostics.please let me know.
                      Also i understand that it will take some time to find the root cause i will be fully prepared and co-operate to provide any information to address the resolution
                      process.Thanks in advance, kindly help me as soon as possible."""
    assistant_output_3 = """{"category": "Technical issues",
                           "tags": ["software behavior", "software compatibility", "Laptop"],
                            "priority": "High",
                             "eta": "Within 5 hours",
                             "response": "Thank you for reaching out to us! We're here to help you with your laptop issue.
                                         We apologize for the inconvenience caused by the erratic software behavior.
                                         Based on your description, it seems like a complex issue that may be related to system or software compatibility.
                                          Our technical team will need more information to diagnose and resolve this issue.
                                          Please provide us with any specific instructions or access to your computer for remote diagnostics."} """
    few_shot_prompt += llama_examples_template.format(
                user_message=user_input_3,
                assistant_message=assistant_output_3
            )

    user_input_4 =""" I spilled water on my touchpad and my touchpad is not getting turned on!!!.Also you help me assess the damage and recover my data?Please help me"""
    assistant_output_4 = """{"category": "Technical issues",
                           "tags": ["water Damage", "Data Recovery", "touchpad"],
                            "priority": "High",
                             "eta": "2-3 business days",
                             "response": "Sorry to hear that you accidentally spilled water on your touchpad! We'll do our best to assist you with assessing the damage and recovering your data.
                                          Please provide us with some more information such as the model of your touchpad and
                                          the extent of the water damage. We will then be able to guide you through the next steps.."} """
    few_shot_prompt += llama_examples_template.format(
                user_message=user_input_4,
                assistant_message=assistant_output_4
            )

    user_input_5 =""" Smartphone is heating up vigourously. It is getting restart automatically. Additionally, Battery charging port is not working. Kindly assist me sooner"""
    assistant_output_5 = """{"category": "Hardware Issues",
                           "tags": ["battery drain", "smartphone"],
                            "priority": "High",
                             "eta": "24 hours",
                             "response": "Thank you for reaching out to us! We're here to help. Based on your description,
                                          it sounds like you might be experiencing a hardware issue with your smartphone's battery charging port.
                                          To assist you further, could you please provide the following information: -
                                          The type of smartphone you are using - The operating system version you have installed -
                                          Any recent software updates or changes made to the device? Once we receive this information,
                                          our technical team will be happy to help diagnose"} """
    few_shot_prompt += llama_examples_template.format(
                user_message=user_input_5,
                assistant_message=assistant_output_5
            )

    user_input_6 =""" On switching on my PC, its throwing blue screen error. I had try the dignostic steps provided by the link in the blue screen, i can't able to fix the issue.
                      Kindly help me in a faster way."""
    assistant_output_6 = """{"category": "Hardware Issues",
                           "tags": ["blue screen of death", "hardware failure"],
                            "priority": "High",
                             "eta": "Within 10 hours",
                             "response": "Thank you for reaching out to us! We're here to help. Sorry to hear that you're experiencing frequent blue screen errors on your PC,
                                          which can be a frustrating issue. Based on the information provided, it appears to be a hardware-related problem. To assist you further,
                                          could you please provide the following information:
                                                1. The make and model of your computer?
                                                2. The operating system version you are using?
                                                3. Any recent software or driver updates installed before the issue started?
                                                4. Have you noticed any specific patterns or triggers that cause the blue screens to occur?
                                                5. Have you run any diagnostic tests or scans on your computer recently?
                                          Once we receive this information, our technical team will be happy to help diagnose
                                          and resolve the hardware issue causing the blue screen errors."} """
    few_shot_prompt += llama_examples_template.format(
                user_message=user_input_6,
                assistant_message=assistant_output_6
            )



    user_input_7 =""" The graphics card in my gaming laptop is not working. On playing the high quality graphic games, the screens are getting freezed.
                      please assist sooner in fixing this hardware issue?."""
    assistant_output_7 = """{"category": "Hardware Issues",
                           "tags": ["graphics card", "gaming laptop"],
                            "priority": "High",
                             "eta": "Within 10 hours",
                             "response": "Thank you for reaching out to us! We're here to help. Based on your description, it sounds like
                                          you might be experiencing a hardware issue with your graphics card. To assist you further, could you please provide the following
                                          information:
                                            1. The make and model of your gaming laptop
                                            2. The type of graphics card installed in your laptop
                                            3. Any recent software updates or changes made to the device?
                                            4. A detailed description of the poor gaming performance you are experiencing (e.g. stuttering, freezing, low frame rates)
                                          Once we receive this information,our technical team will be happy to help diagnose and fix the issue."} """
    few_shot_prompt += llama_examples_template.format(
                user_message=user_input_7,
                assistant_message=assistant_output_7
            )


    return few_shot_prompt

In [633]:
few_shot_prompt = create_few_shot_prompt(system_message)

In [634]:
few_shot_prompt

'<s>[INST] You are a Customer Support Assistant for ABCDE tech.co.in.\nYou will be provided with user complaints. User complaints will be delimited by triple backticks, that is, ```.\nYour task is to understand the support ticket text completely and generate a response to all the user complaints. Please process the user\'s complaint,\nidentify its sentiment, and respond appropriately, always maintaining a polite and helpful tone. If possible, provide relevant information or\ntroubleshooting steps.\n\nIf the sentiment is negative or critical, assure the user that their concern is taken seriously, and a representative will address it as soon as\npossible.\nIf the sentiment is positive or neutral, acknowledge the issue and offer any immediate assistance or guidance available.\n\noutput should be in the form of a JSON object for all the given user complaints. Go through the user\'s feedback step by step, and generate a structured output for further analysis by the customer support assistan

In [636]:
def generate_few_shot_llama_response(user_message):
    #create prompt
    prompt=create_few_shot_prompt(system_message)

    user_input = llama_prediction_template.format(
            user_message=user_message,
            assistant_message=''
        )

    # Generate a response from the LLaMA model
    response = lcpp_llm(
        prompt=prompt+user_input,
        max_tokens=512,
        temperature=0,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        echo=False
    )

    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    return response_text

# Checking Sample Response:

In [595]:
user_input="""I hope this message finds you well. I am writing to report a perplexing issue I've encountered with my work computer in recent days. \nThe problem seems to involve a combination of unusual software behavior and unexpected data loss.\nOver the past week, I've observed that certain software applications on my computer have been \nbehaving erratically. For example, some applications freeze randomly, while others exhibit unexplained crashes. Additionally, \nI've noticed that some files and documents that were previously saved on my desktop have mysteriously \ndisappeared. These issues are causing significant disruptions to my daily tasks and workflow.\nWhile I don't have specific instructions on how to resolve this complex problem,\nI suspect there may be an underlying issue with the system or software compatibility. \nI kindly request your expertise and assistance in thoroughly diagnosing this intricate problem. \nYour insights and guidance would be greatly appreciated. If you require any additional information \nor access to my computer for remote diagnostics, please do not hesitate to let me know.\nI understand that this issue may require some time and effort to address, and I am fully prepared \nto cooperate and provide any necessary information to facilitate the resolution process. Thank you in advance for your attention to this matter, and I look forward to your response."""

user_input = user_input.replace("\n","")

In [596]:
user_input

"I hope this message finds you well. I am writing to report a perplexing issue I've encountered with my work computer in recent days. The problem seems to involve a combination of unusual software behavior and unexpected data loss.Over the past week, I've observed that certain software applications on my computer have been behaving erratically. For example, some applications freeze randomly, while others exhibit unexplained crashes. Additionally, I've noticed that some files and documents that were previously saved on my desktop have mysteriously disappeared. These issues are causing significant disruptions to my daily tasks and workflow.While I don't have specific instructions on how to resolve this complex problem,I suspect there may be an underlying issue with the system or software compatibility. I kindly request your expertise and assistance in thoroughly diagnosing this intricate problem. Your insights and guidance would be greatly appreciated. If you require any additional infor

In [597]:
response = generate_few_shot_llama_response(user_input)

Llama.generate: prefix-match hit


In [598]:
print(response)



{ "category": "Software Issues",
"tags": ["software behavior", "data loss"],
"priority": "High",
"eta": "Within 10 hours",
"response": "Thank you for reaching out to us with your complex issue. We understand the significance of resolving this matter as soon as possible, and we're here to help. Based on your description, it seems like there might be a compatibility issue between certain software applications and your system, resulting in erratic behavior and data loss. To assist you further, could you please provide us with the following information:

1. A detailed list of the software applications that are exhibiting erratic behavior and crashes.
2. The version numbers of the operating system and all installed software.
3. Any specific error messages or notifications you have received related to the issue.
4. Your desktop files and documents that have disappeared, including their location and size.

Once we receive this information, our technical team will conduct a thorough diagnosi

# Loading the Dataset

In [637]:
user_complaints_fw_sht_df = pd.read_csv('Support_ticket_text_data_mid_term.csv')

In [638]:
user_complaints_fw_sht_df.head(15)

,support_tick_id,support_ticket_text
0,ST2023-006,"My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly."
1,ST2023-007,"Urgent help required! My laptop refuses to start, and I have a crucial presentation scheduled for tomorrow. I've attempted a restart, but it hasn't worked. Please provide immediate assistance to resolve this hardware issue"
2,ST2023-008,"I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?"
3,ST2023-009,"Despite being in close proximity to my Wi-Fi router, the signal remains persistently weak in my home. This issue has been ongoing, and I need assistance troubleshooting it. Please help me resolve the weak Wi-Fi signal problem."
4,ST2023-010,"My smartphone battery is draining rapidly, even with minimal use. Can you help me identify and rectify this battery issue?"
5,ST2023-011,I'm locked out of my online banking account and need immediate assistance in resetting my password for an urgent transaction.
6,ST2023-012,"My computer's performance is sluggish, severely impacting my work. I need help optimizing it to regain productivity."
7,ST2023-013,"I'm experiencing a recurring blue screen error on my PC, causing frequent crashes. Help me diagnose and resolve this hardware issue."
8,ST2023-014,My external hard drive isn't being recognized by my computer. I need assistance recovering vital data stored on it.
9,ST2023-015,"The graphics card in my gaming laptop seems to be malfunctioning, causing poor gaming performance. Can you assist in diagnosing and fixing this hardware issue?"


In [639]:
user_complaints_fw_sht_df.shape

(21, 2)

# Applying the lLama model with few shot promts and user complaints

In [640]:
# Constructing the response for all the records/complaints in the input dataset using the few shot prompts.

user_complaints_fw_sht_df['llama_response'] = user_complaints_fw_sht_df['support_ticket_text'].apply(lambda x: generate_few_shot_llama_response(x.replace("\n","")))

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


In [641]:
#Response in Json object format
user_complaints_fw_sht_df

,support_tick_id,support_ticket_text,llama_response
0,ST2023-006,"My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly.","{\n""category"": ""Internet Connectivity Issues"",\n""tags"": [""slow internet"", ""frequent disconnections""],\n""priority"": ""High"",\n""eta"": ""Within 4 hours"",\n""response"": ""Thank you for reaching out to us! We apologize for the inconvenience caused by the slow internet speeds and frequent disconnections. Our technical team will be happy to assist you in resolving this issue promptly. To help us investigate, could you please provide the following information:\n\n1. Your account details (account name/number or phone number)\n2. The type of internet connection you have (e.g., DSL, cable, fiber)\n3. The speed you are currently experiencing\n4. Any recent changes or updates to your network or computer\n5. A detailed description of the disconnections and any error messages you may have received\n\nOnce we receive this information, our technical team will start investigating the issue and work towards resolving it as soon as possible."""
1,ST2023-007,"Urgent help required! My laptop refuses to start, and I have a crucial presentation scheduled for tomorrow. I've attempted a restart, but it hasn't worked. Please provide immediate assistance to resolve this hardware issue","{ ""category"": ""Hardware Issues"",\n""tags"": [""laptop"", ""startup"", ""presentation""],\n""priority"": ""Critical"",\n""eta"": ""Immediate"",\n""response"": ""Sorry to hear that your laptop is not starting! 😱 We understand the urgency of the situation and will do our best to assist you. Can you please provide us with some more information about the issue? 🤔 Have you tried booting in safe mode or using a different power source? Additionally, can you tell us the make and model of your laptop?"" }"
2,ST2023-008,"I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?","{\n""category"": ""Data Recovery"",\n""tags"": [""data loss"", ""documents""],\n""priority"": ""High"",\n""eta"": ""Within 24 hours"",\n""response"": ""Sorry to hear that you've accidentally deleted essential work documents! We understand the importance of recovering your data as soon as possible. To assist you with the data recovery process, we request that you follow these steps:\n\n1. Stop using the device immediately: Any further activity on the device may overwrite or corrupt the deleted files, making them irretrievable.\n2. Keep the device powered on: Do not turn off the device until the recovery process is complete.\n3. Provide us with the following information:\n\t* The type of device (e.g., laptop, desktop, smartphone)\n\t* The operating system installed on the device\n\t* A detailed description of the documents you have deleted\n\t* Any specific software or applications you have used recently\n4. Wait for further instructions from our team. We will guide you through the data recovery process and provide any necessary tools or software to assist in the recovery of your essential work documents."""
3,ST2023-009,"Despite being in close proximity to my Wi-Fi router, the signal remains persistently weak in my home. This issue has been ongoing, and I need assistance troubleshooting it. Please help me resolve the weak Wi-Fi signal problem.","{\n""category"": ""Wi-Fi Issues"",\n""tags"": [""weak signal"", ""proximity""],\n""priority"": ""High"",\n""eta"": ""Within 10 hours"",\n""response"": ""Thank you for reaching out to us! We're here to help. Sorry to hear that you're experiencing a persistent weak Wi-Fi signal issue in your home, despite being close to the router. To assist you further, could you please provide t

In [642]:
user_complaints_ext_df = user_complaints_fw_sht_df.copy(deep=True)

In [643]:
# Function to parse JSON data and extract key-value pairs
import json

def extract_json_data(json_str):
    try:
        # Attempt to load the JSON string
        json_str = json_str.replace("\n","")
        json_str = json_str.replace("\t*","")
        x = json_str.index("""{""")
        print(json_str)
        data_dict = json.loads(json_str[x:])
        return data_dict
    except json.JSONDecodeError as e:
        if json_str.endswith('}') or json_str.endswith('},'):
            # JSON string already ends with a closing brace or a closing brace preceded by a comma
            # Nothing more to add, return empty dictionary
            return {}
        elif json_str.endswith('"'):
            # JSON string ends with a ", add a closing brace
            json_str += '}'
        elif json_str.endswith(','):
            # JSON string ends with a comma, add a closing brace
            json_str += '}'
        else:
            # JSON string doesn't end with a comma or a closing brace, add both
            json_str += ',}'
        try:
            # Attempt to load the JSON string again
            data_dict = json.loads(json_str)
            return data_dict
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {e}")
            return {}

In [644]:
user_complaints_ext_df['lLama_response_parsed'] = user_complaints_ext_df['llama_response'].apply(extract_json_data)

{"category": "Internet Connectivity Issues","tags": ["slow internet", "frequent disconnections"],"priority": "High","eta": "Within 4 hours","response": "Thank you for reaching out to us! We apologize for the inconvenience caused by the slow internet speeds and frequent disconnections. Our technical team will be happy to assist you in resolving this issue promptly. To help us investigate, could you please provide the following information:1. Your account details (account name/number or phone number)2. The type of internet connection you have (e.g., DSL, cable, fiber)3. The speed you are currently experiencing4. Any recent changes or updates to your network or computer5. A detailed description of the disconnections and any error messages you may have receivedOnce we receive this information, our technical team will start investigating the issue and work towards resolving it as soon as possible."
 { "category": "Hardware Issues","tags": ["laptop", "startup", "presentation"],"priority": "C

# Parsed lLama response:

In [645]:
user_complaints_ext_df

,support_tick_id,support_ticket_text,llama_response,lLama_response_parsed
0,ST2023-006,"My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly.","{\n""category"": ""Internet Connectivity Issues"",\n""tags"": [""slow internet"", ""frequent disconnections""],\n""priority"": ""High"",\n""eta"": ""Within 4 hours"",\n""response"": ""Thank you for reaching out to us! We apologize for the inconvenience caused by the slow internet speeds and frequent disconnections. Our technical team will be happy to assist you in resolving this issue promptly. To help us investigate, could you please provide the following information:\n\n1. Your account details (account name/number or phone number)\n2. The type of internet connection you have (e.g., DSL, cable, fiber)\n3. The speed you are currently experiencing\n4. Any recent changes or updates to your network or computer\n5. A detailed description of the disconnections and any error messages you may have received\n\nOnce we receive this information, our technical team will start investigating the issue and work towards resolving it as soon as possible.""","{'category': 'Internet Connectivity Issues', 'tags': ['slow internet', 'frequent disconnections'], 'priority': 'High', 'eta': 'Within 4 hours', 'response': 'Thank you for reaching out to us! We apologize for the inconvenience caused by the slow internet speeds and frequent disconnections. Our technical team will be happy to assist you in resolving this issue promptly. To help us investigate, could you please provide the following information:1. Your account details (account name/number or phone number)2. The type of internet connection you have (e.g., DSL, cable, fiber)3. The speed you are currently experiencing4. Any recent changes or updates to your network or computer5. A detailed description of the disconnections and any error messages you may have receivedOnce we receive this information, our technical team will start investigating the issue and work towards resolving it as soon as possible.'}"
1,ST2023-007,"Urgent help required! My laptop refuses to start, and I have a crucial presentation scheduled for tomorrow. I've attempted a restart, but it hasn't worked. Please provide immediate assistance to resolve this hardware issue","{ ""category"": ""Hardware Issues"",\n""tags"": [""laptop"", ""startup"", ""presentation""],\n""priority"": ""Critical"",\n""eta"": ""Immediate"",\n""response"": ""Sorry to hear that your laptop is not starting! 😱 We understand the urgency of the situation and will do our best to assist you. Can you please provide us with some more information about the issue? 🤔 Have you tried booting in safe mode or using a different power source? Additionally, can you tell us the make and model of your laptop?"" }","{'category': 'Hardware Issues', 'tags': ['laptop', 'startup', 'presentation'], 'priority': 'Critical', 'eta': 'Immediate', 'response': 'Sorry to hear that your laptop is not starting! 😱 We understand the urgency of the situation and will do our best to assist you. Can you please provide us with some more information about the issue? 🤔 Have you tried booting in safe mode or using a different power source? Additionally, can you tell us the make and model of your laptop?'}"
2,ST2023-008,"I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?","{\n""category"": ""Data Recovery"",\n""tags"": [""data loss"", ""documents""],\n""priority"": ""High"",\n""eta"": ""Within 24 hours"",\n""response"": ""Sorry to hear that you've accidentally deleted essential work documents! We understand the importance of recovering your data as soon as possible. To assist you with the data recovery

# Preparing the final dataset with the parsed Json response as below:

In [646]:
final_customer_df = pd.concat([user_complaints_ext_df[["support_tick_id","support_ticket_text"]], pd.json_normalize(user_complaints_ext_df['lLama_response_parsed'])], axis=1)

In [647]:
final_customer_df

,support_tick_id,support_ticket_text,category,tags,priority,eta,response
0,ST2023-006,"My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly.",Internet Connectivity Issues,"[slow internet, frequent disconnections]",High,Within 4 hours,"Thank you for reaching out to us! We apologize for the inconvenience caused by the slow internet speeds and frequent disconnections. Our technical team will be happy to assist you in resolving this issue promptly. To help us investigate, could you please provide the following information:1. Your account details (account name/number or phone number)2. The type of internet connection you have (e.g., DSL, cable, fiber)3. The speed you are currently experiencing4. Any recent changes or updates to your network or computer5. A detailed description of the disconnections and any error messages you may have receivedOnce we receive this information, our technical team will start investigating the issue and work towards resolving it as soon as possible."
1,ST2023-007,"Urgent help required! My laptop refuses to start, and I have a crucial presentation scheduled for tomorrow. I've attempted a restart, but it hasn't worked. Please provide immediate assistance to resolve this hardware issue",Hardware Issues,"[laptop, startup, presentation]",Critical,Immediate,"Sorry to hear that your laptop is not starting! 😱 We understand the urgency of the situation and will do our best to assist you. Can you please provide us with some more information about the issue? 🤔 Have you tried booting in safe mode or using a different power source? Additionally, can you tell us the make and model of your laptop?"
2,ST2023-008,"I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?",Data Recovery,"[data loss, documents]",High,Within 24 hours,"Sorry to hear that you've accidentally deleted essential work documents! We understand the importance of recovering your data as soon as possible. To assist you with the data recovery process, we request that you follow these steps:1. Stop using the device immediately: Any further activity on the device may overwrite or corrupt the deleted files, making them irretrievable.2. Keep the device powered on: Do not turn off the device until the recovery process is complete.3. Provide us with the following information: The type of device (e.g., laptop, desktop, smartphone) The operating system installed on the device A detailed description of the documents you have deleted Any specific software or applications you have used recently4. Wait for further instructions from our team. We will guide you through the data recovery process and provide any necessary tools or software to assist in the recovery of your essential work documents."
3,ST2023-009,"Despite being in close proximity to my Wi-Fi router, the signal remains persistently weak in my home. This issue has been ongoing, and I need assistance troubleshooting it. Please help me resolve the weak Wi-Fi signal problem.",Wi-Fi Issues,"[weak signal, proximity]",High,Within 10 hours,"Thank you for reaching out to us! We're here to help. Sorry to hear that you're experiencing a persistent weak Wi-Fi signal issue in your home, despite being close to the router. To assist you further, could you please provide the following information:1. The make and model of your Wi-Fi router?2. The type of device(s) you are using to connect to the Wi-Fi network?3. The distance between your devices and the router?4. Any obstructions or physical barriers between the devices and the router?5. Have you tried moving the router to a different location in your home?6. Are there any other wireless networks nearby that could be in

# Q7 - Observations and Insights:
   - On checking the above response, Llama 2 13 b model is good in understanding the complaints context and issues given by the users. It is capable of spliting the given context into Categories, assigning the tags for the user complaints, urgency/priority to solve the issue with proper and realistic ETA's.
   - Model is also capable of providing the first line response based on the sentiment provided in the ticket.
   -  On using the Zero shot prompt there are few responses are not in the asked format.
   - But when we have given a few samples as input to the model, it is capable of replying in the requested format.
   - Model is keen in analysing the context and asking for more information about the issue.
   - This model can be itegrate with streamlit as a reponsive chat bot to user and also can efficiently do the support ticket categorization. Based on the category and ticket priority we can automate the ticket to assign to different Level of engineers to solve the issues with proactiveness and helps in Engineering resource management.
   - On a General note,The Llama 2 model, as we can see, is able to produce high-quality outputs that would be considered on par with the quality of OpenAI's GPT-3.5, highlighting the progress that open-source LLMs have been able to make.
